In [1]:

import os
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI
from operator import itemgetter
import time

# OpenAI API 키 설정 (사용자가 직접 설정해야 함)
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"

class MemoryBot:
    def __init__(self, model_name="gpt-5-nano", temperature=0.5):
        """메모리 기능이 있는 챗봇 초기화"""

        # LLM 모델 설정
        self.llm = ChatOpenAI(model=model_name, temperature=temperature)

        # 프롬프트 템플릿 설정
        self.prompt = ChatPromptTemplate([
            ('system', '당신은 도움이 되는 친근한 AI 어시스턴트입니다. 사용자와의 이전 대화 내용을 기억하며 자연스럽게 대화하세요.'),
            MessagesPlaceholder(variable_name='chat_history'),  # 기존 채팅 내역 주입
            ('human', '{input}'),
        ])

        # 메모리 설정
        self.memory = ConversationBufferMemory(
            return_messages=True, 
            memory_key='chat_history'
        )

        # 실행 체인 구성
        self.runnable = RunnablePassthrough.assign(
            chat_history=RunnableLambda(self.memory.load_memory_variables) |
            itemgetter('chat_history')
        )

        self.chain = self.runnable | self.prompt | self.llm

        print("🤖 기억하는 챗봇이 준비되었습니다!")
        print("💡 ('quit', '정지', '그만', 'exit') 중 아무거나 입력하면 대화가 종료됩니다.")
        print("=" * 50)

    def chat(self, user_input):
        """사용자 입력을 받아 AI 응답 생성 및 메모리 저장"""
        try:
            # AI 응답 생성
            response = self.chain.invoke({'input': user_input})
            ai_response = response.content

            # 대화 내용을 메모리에 저장
            self.memory.save_context(
                {'human': user_input},
                {'ai': ai_response}
            )

            return ai_response

        except Exception as e:
            return f"죄송합니다. 오류가 발생했습니다: {str(e)}"

    def get_memory(self):
        """현재 저장된 메모리 내용 확인"""
        return self.memory.load_memory_variables({})

    def clear_memory(self):
        """메모리 내용 초기화"""
        self.memory.clear()
        print("🧹 메모리가 초기화되었습니다.")

    def start_conversation(self):
        """대화 시작"""
        print("👋 안녕하세요! 무엇을 도와드릴까요?\n")

        while True:
            try:
                # 사용자 입력 받기
                user_input = input("😊 You: ").strip()

                # 종료 명령 확인
                if user_input.lower() in ('quit', '정지', '그만', 'exit', 'q'):
                    print("\n👋 대화를 종료합니다. 좋은 하루 되세요!")
                    break

                # 빈 입력 처리
                if not user_input:
                    print("❓ 메시지를 입력해주세요.")
                    continue

                # 특별 명령어 처리
                if user_input.lower() == '/memory':
                    memory_content = self.get_memory()
                    print(f"\n🧠 현재 메모리 내용:")
                    for msg in memory_content.get('chat_history', []):
                        role = "사용자" if msg.type == "human" else "AI"
                        print(f"   {role}: {msg.content}")
                    print()
                    continue

                if user_input.lower() == '/clear':
                    self.clear_memory()
                    continue

                # AI 응답 생성
                print("\n🤖 Bot: ", end="")
                ai_response = self.chat(user_input)
                print(ai_response)
                print("-" * 50)

            except KeyboardInterrupt:
                print("\n\n👋 대화가 중단되었습니다.")
                break
            except Exception as e:
                print(f"\n❌ 오류 발생: {e}")

def main():
    """메인 실행 함수"""
    print("🚀 기억하는 AI 챗봇을 시작합니다!")
    print("\n📝 사용 가능한 명령어:")
    print("   /memory - 현재 대화 기록 확인")
    print("   /clear  - 메모리 초기화") 
    print("   quit, 정지, 그만, exit - 프로그램 종료")
    print()

    try:
        # 챗봇 인스턴스 생성 및 대화 시작
        bot = MemoryBot()
        bot.start_conversation()

    except Exception as e:
        print(f"❌ 챗봇 초기화 중 오류 발생: {e}")
        print("💡 OpenAI API 키가 설정되어 있는지 확인해주세요.")

if __name__ == "__main__":
    main()


🚀 기억하는 AI 챗봇을 시작합니다!

📝 사용 가능한 명령어:
   /memory - 현재 대화 기록 확인
   /clear  - 메모리 초기화
   quit, 정지, 그만, exit - 프로그램 종료



C:\Users\tipay\AppData\Local\Temp\ipykernel_17460\2258323909.py:27: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  self.memory = ConversationBufferMemory(


🤖 기억하는 챗봇이 준비되었습니다!
💡 ('quit', '정지', '그만', 'exit') 중 아무거나 입력하면 대화가 종료됩니다.
👋 안녕하세요! 무엇을 도와드릴까요?


🤖 Bot: 안녕, 팽이야! 반가워요. 어떤 걸 같이 해볼까? 선택지 몇 가지를 제안해줄게.

- 잘 도는 법 배우기: 팽이가 오래 도는 간단한 기술 팁
- 팽이 원리 쉽게 이해하기: 자이로스코프 같은 원리 간단 설명
- 디자인 아이디어: 무게 배치나 재료, 색상으로 멋진 팽이 만들기
- 짧은 퀴즈나 팽이 재미 facts

혹시 위 중에 어떤 게 좋을지 말해줘. 아니면 원하던 주제가 있다면 알려줘!
--------------------------------------------------

🤖 Bot: 좋아하네! 닉네임으로도 멋진 분위기 만들 수 있어. 아래 중에서 원하는 걸 골라줘, 바로 시작할게.

- 닉네임 배경 이야기 만들기: 왜 '팽이'를 택했는지 짧은 백스토리 만들어주기
- 팽이 테마 짧은 창작 글: 팽이가 주인공인 200~300자짜리 이야기
- 간단 퀴즈 또는 게임 아이디어: 예/아니오 퀴즈나 5문제짜리 짧은 퍼즐
- 관심사 맞춤 아이디어: 음악, 게임, 코딩 등 네가 좋아하는 주제로 맞춤 콘텐츠 제시

다른 아이디어가 있으면 말씀해줘! 어떤 방향으로 시작할지 알려주면 바로 진행할게.
--------------------------------------------------

🤖 Bot: 좋아, 너무 무겁지 않게 가볼게! 가볍게 대화할 수 있는 몇 가지 옵션이야. 마음에 드는 걸 골라줘, 바로 시작할게.

- 한 줄 팽이 모험 이야기 듣기: 예시로도 바로 들려줄 수 있어. 예시) 오늘도 바람을 타고 굴러간 팽이는 골목 끝에서 아이의 미소를 만난 뒤 다시 한 바퀴를 돌며 인사를 했지.
- 팽이 짧은 농담 한 줄: 예) 팽이가 친구들에게 자랑하길 "난 원만한 친구야. 언제나 한 바퀴 돌고 돌아와 약속 지켜주지!" 
- 오늘의 가벼운 음악 추천 1곡: 분위기만 알려주면 간단히 한 곡